# Vertical sections of radiousounding variables (Level 1 data) and corresponding TermoSalinoGraph data from the R/Vs.
agostino.meroni@gmail.com

In [ ]:
import xarray as xr
import pydap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import colors 
from matplotlib import cm
import matplotlib
from glob import glob


To protect the login credentials of Aeris, they need to be entered by the user into the credentials.json and called to this script using the cell below.

In [ ]:
credentials = json.loads(open('credentials.json').read()) #don't forget to add the login credentials to the credentials.json file!

username = credentials['username']
password = credentials['pass']

In [ ]:
# Example: how to read radar data with pydap.
url = 'http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SATELLITES/GOES-E/2km_10min/2020/2020_02_03/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20200342040115_BARBADOS-2KM-FD.level2.nc'
store = xr.backends.PydapDataStore.open(url)
ds = xr.open_dataset(store)

In [ ]:
ds.cloud_fraction.plot()

In [ ]:
# You can download the Meteor DSHIP data with wget from the terminal with this line:
# wget -r -l1 --no-parent -A.dat https://observations.ipsl.fr/aeris/eurec4a-data/SHIPS/RV-METEOR/DSHIP/

In [ ]:
# After downloading some TSG file from the R/V Atalante locally 
# (from here https://observations.ipsl.fr/aeris/eurec4a/#/), open one of them and plot the SST.

from glob import glob

path2tsg = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/atalante/tsg_aeris/'
filenames = [x for x in glob(path2tsg + '*.nc')]
print(filenames)

ds = xr.open_dataset(filenames[0])
ds.SSJT.plot()
plt.show()

In [ ]:
# This uses OpenDAP to read the TSG file of the Atalante on the 2nd February.
url = 'http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/INSITU/TSTS/FNCM/2020/20200202/FNCM_TSTS_20200202.nc'
store = xr.backends.PydapDataStore.open(url)
ds = xr.open_dataset(store)
ds.SSJT.plot()


In [ ]:
# Open all the radiosounding files in the 2020_02_02 folder from the Atalante (pydap) and plot a 
# vertical section of the temperature.

listoffiles = ['ATL_SoundingAscentProfile_Atalante_20200203_233642.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_224034.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_214956.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_210007.nc',
               'ATL_SoundingAscentProfile_Atalante_20200202_201504.nc',
               'ATL_SoundingAscentProfile_Atalante_20200202_193236.nc']

for k in range(len(listoffiles)):
    url = 'http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SHIPS/RV-ATALANTE/RADIOSOUNDING/METEOMODEM/NetCDF/2020/2020_02_02/'+listoffiles[k]
    store = xr.backends.PydapDataStore.open(url)
    rs = xr.open_dataset(store)
    # Plot a scatterplot on a time-altitude section of temperature
    #xr.plot.scatter(rs,x='flight_time',y='altitude',hue='temperature')
    plt.scatter(rs.flight_time.values,rs.altitude.values,c=rs.temperature.values)



In [ ]:
# Radiosounding from R/V Meteor using OpenDAP.

plt.figure(figsize=(10,5))

listoffiles = ['M161_SoundingDescentProfile_FS_Meteor_20200120_2017.nc',
               'M161_SoundingDescentProfile_FS_Meteor_20200120_1605.nc',
               'M161_SoundingDescentProfile_FS_Meteor_20200120_1216.nc']#,
#               'M161_SoundingDescentProfile_FS_Meteor_20200120_0830.nc',
#               'M161_SoundingDescentProfile_FS_Meteor_20200120_0425.nc',
#               'M161_SoundingDescentProfile_FS_Meteor_20200120_0023.nc', 
#               'M161_SoundingAscentProfile_FS_Meteor_20200120_2244.nc',
#               'M161_SoundingAscentProfile_FS_Meteor_20200120_1844.nc',
#               'M161_SoundingAscentProfile_FS_Meteor_20200120_1444.nc',
#               'M161_SoundingAscentProfile_FS_Meteor_20200120_1045.nc',
#               'M161_SoundingAscentProfile_FS_Meteor_20200120_0645.nc',
#               'M161_SoundingAscentProfile_FS_Meteor_20200120_0245.nc']

for name in listoffiles:
    url = 'http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SHIPS/RV-METEOR/radiosondes/2020/NC/'+name
    store = xr.backends.PydapDataStore.open(url)
    rs = xr.open_dataset(store)

    sc = plt.scatter(rs.flight_time.values,rs.altitude.values,c=rs.temperature.values)
    
plt.colorbar()
plt.ylabel('Altitude [m]')
plt.xlabel('Time of flight [UTC?]')

In [ ]:
# Read a DSHIP file which is downloaded locally.
path2dship = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/meteor/dship/'
filename = 'RV-METEOR_DSHIP_1Hz_20200120.dat'

df = pd.read_table(path2dship+filename, encoding='ISO-8859-1')
#df['WEATHER.PBWWI.WaterTempStarboard']

# Remove the first two entries (that are not numbers) and convert to double.
timestamp = df['SYS.CALC.Timestamp'][2:-1].to_numpy(dtype='double') #[1 sec]
#plt.plot(timestamp)

temp_p =df['WEATHER.PBWWI.WaterTempPort'][2:-1].to_numpy(dtype='double')
plt.plot(temp_p)
temp_p
#This field is empty!!

temp_s =df['WEATHER.PBWWI.WaterTempStarboard'][2:-1].to_numpy(dtype='double')
plt.plot(temp_s)
temp_s
#This field is empty, as well!!

In [ ]:
# Example of radiosouding from the RON/BROWN.

#url = 'http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SHIPS/RV-RONALDHBROWN/radiosonde/RHB_SoundingDescentProfile_15N54W_20200207_1945.nc'
url = 'http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SHIPS/RV-RONALDHBROWN/radiosonde/RHB_SoundingDescentProfile_15N54W_20200204_0813.nc'
store = xr.backends.PydapDataStore.open(url)
rs = xr.open_dataset(store)
rs

# Read TSG file and RS files saved locally: Meteor 6-8 February 2020

In [ ]:
# Read all the radiosounding files on the 7th February.
doi = '20200208' # Select the date of interest in the format 'YYYYMMDD'

path2rs = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/RS_level_1/MET/Vaisala/'

from glob import glob

listoffiles = [x for x in glob(path2rs + '*.nc')]
#print(listoffiles)

# Select only the files for the date of interest.
match = [s for s in listoffiles if doi in s]
#print(match)

In [ ]:
# Read now the TSG.

path2tsg = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/meteor/tsg_aeris/'

listoffiles_tsg = [x for x in glob(path2tsg + '*.nc')]
filename = [s for s in listoffiles_tsg if doi in s]
print(filename)
#filename = 'DBBH_TSTS_20200207.nc' # It contains the date

tsg = xr.open_dataset(filename[0])

In [ ]:
# Make the figure, plot the vertical section of RS data in the first subplot and the time series of the 
# TSG in the second subplot.

fig, ax = plt.subplots(2,1, figsize=(10, 6), sharex=True, gridspec_kw={"height_ratios": [2, 1]})
ax = ax.ravel()

cmap = cm.get_cmap('plasma',10) # Set here the colormap and the number of colors.
kk = 0
ta_surf = np.zeros(len(match))
time_surf = np.zeros(len(match))

for name in match:
   # print(name)
    rs = xr.open_dataset(name)
    ta = rs.ta.values-273.15
    sc = ax[0].scatter(rs.flight_time.values, rs.alt.values, c=ta, cmap=cmap, vmin=-75, vmax=25)
    ta_surf[kk] = ta[0,0]
    time_surf[kk] = rs.flight_time.values[0,0]
    #print(ta[0,0])
    kk += 1

#normalize = colors.Normalize(vmin = ta.min(), vmax = ta.max())
#cbar = fig.colorbar(matplotlib.cm.ScalarMappable(norm=normalize, cmap=cmap),ax=ax,
#                    orientation='vertical',label='Air temperature')

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.15, 0.02, 0.7]) # Set the position and dimension of the colorbar.
fig.colorbar(sc, shrink=0.5, cax=cbar_ax, label='Air temperature [°C]')

ax[1].plot(tsg.TIME, tsg.SSJT)
#ax[0].set_ylim(0,2000)
#ax[1].plot(time_surf,ta_surf) # It complains about the time format...
#ax[1].set_ylim(-5,0)
ax[0].set_ylabel('Height [m]')
ax[1].set_ylabel('Sea Temperature [°C]')
ax[1].set_xlabel('Time [UTC]')
plt.subplots_adjust(wspace=0, hspace=0)

plt.savefig('meteor_rs_temp_' + doi + '.png')

In [ ]:
# This is a useful link for the data
# https://observations.ipsl.fr/aeris/eurec4a-data/

In [ ]:
# Focus on the first 2 km of the atmosphere.

fig, ax = plt.subplots(2,1, figsize=(10, 6), sharex=True, gridspec_kw={"height_ratios": [2, 1]})
ax = ax.ravel()

cmap = cm.get_cmap('plasma',15) # Set here the colormap and the number of colors.
kk = 0
ta_surf = np.zeros(len(match))
time_surf = np.empty(len(match), dtype='datetime64[ns]')

for name in match:
   # print(name)
    rs = xr.open_dataset(name)
    ta = rs.ta.values-273.15
    sc = ax[0].scatter(rs.flight_time.values, rs.alt.values, c=ta, cmap=cmap, vmin=10, vmax=25)
    if ta[0,0]>20: # Take only the first value of the ascending RS: we want the surface air temperature.
        ta_surf[kk] = ta[0,0]
        time_surf[kk] = rs.flight_time[0,0].values
    else:
        ta_surf[kk] = np.nan
        time_surf[kk] = np.datetime64('nat')
    kk += 1
#print(ta_surf)
#print(time_surf)

#normalize = colors.Normalize(vmin = ta.min(), vmax = ta.max())
#cbar = fig.colorbar(matplotlib.cm.ScalarMappable(norm=normalize, cmap=cmap),ax=ax,
#                    orientation='vertical',label='Air temperature')

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.15, 0.02, 0.7]) # Set the position and dimension of the colorbar.
fig.colorbar(sc, shrink=0.5, cax=cbar_ax, label='Air temperature [°C]')

ax[1].plot(tsg.TIME, tsg.SSJT)
ax[0].set_ylim(0,2000)
ax[1].plot(time_surf,ta_surf,'o')
#ax[1].set_ylim(-5,0)
ax[0].set_ylabel('Height [m]')
ax[1].set_ylabel('Temperature [°C]')
ax[1].set_xlabel('Time [UTC]')
ax[1].legend(['Sea','Air'])
plt.subplots_adjust(wspace=0, hspace=0)

plt.savefig('meteor_rs_temp_low_' + doi + '.png')

In [ ]:
# Read the ceilometer files for the same day and plot the relative humidity field.

path2ceil = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/meteor/ceilometer/'

listoffiles_ceil = [x for x in glob(path2ceil + '*.nc')]
filename_ceil = [s for s in listoffiles_ceil if doi in s]
print(filename_ceil)
#filename = 'DBBH_TSTS_20200207.nc' # It contains the date

ceil = xr.open_dataset(filename_ceil[0])
#ceil.cbh.sel(layer=1).plot() # The cloud base height has three levels (layer).
#plt.show()
#ceil

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10, 6), sharex=True, gridspec_kw={"height_ratios": [2, 1]})
ax = ax.ravel()

cmap = cm.get_cmap('viridis',8).reversed() # Set here the colormap and the number of colors.
kk = 0
ta_surf = np.zeros(len(match))
time_surf = np.empty(len(match), dtype='datetime64[ns]')

for name in match:
   # print(name)
    rs = xr.open_dataset(name)
    ta = rs.ta.values-273.15
    sc = ax[0].scatter(rs.flight_time.values, rs.alt.values, c=rs.rh.values, cmap=cmap, vmin=0.1, vmax=0.9)
    if ta[0,0]>20: # Take only the first value of the ascending RS: we want the surface air temperature.
        ta_surf[kk] = ta[0,0]
        time_surf[kk] = rs.flight_time[0,0].values
    else:
        ta_surf[kk] = np.nan
        time_surf[kk] = np.datetime64('nat')
    kk += 1
#print(ta_surf)
#print(time_surf)

# Add the ceilometer clod base height.
ax[0].plot(ceil.time.values, ceil.cbh.sel(layer=1),'.r')
ax[0].legend(['1st layer cloud base height'], loc=2)

#normalize = colors.Normalize(vmin = ta.min(), vmax = ta.max())
#cbar = fig.colorbar(matplotlib.cm.ScalarMappable(norm=normalize, cmap=cmap),ax=ax,
#                    orientation='vertical',label='Air temperature')

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.15, 0.02, 0.7]) # Set the position and dimension of the colorbar.
fig.colorbar(sc, shrink=0.5, cax=cbar_ax, label='Relative Humidity [1]')

ax[1].plot(tsg.TIME, tsg.SSJT)
ax[0].set_ylim(0,3000)
ax[1].plot(time_surf,ta_surf,'o')
#ax[1].set_ylim(-5,0)
ax[0].set_ylabel('Height [m]')
ax[1].set_ylabel('Temperature [°C]')
ax[1].set_xlabel('Time [UTC]')
ax[1].legend(['Sea','Air'])
plt.subplots_adjust(wspace=0, hspace=0)

plt.savefig('meteor_rs_RH_low_' + doi + '.png')

In [ ]:
# Setup the figure: temperature in the lower 3000 m.

fig, ax = plt.subplots(2,1, figsize=(15, 6), sharex=True, gridspec_kw={"height_ratios": [2, 1]})
ax = ax.ravel()

cmap = cm.get_cmap('plasma',15) # Set here the colormap and the number of colors.
kk = 0
ta_surf = np.zeros(len(match))
time_surf = np.empty(len(match), dtype='datetime64[ns]')

# Open all the three days.

# Radiosounding
path2rs = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/RS_level_1/MET/Vaisala/'
listoffiles = [x for x in glob(path2rs + '*.nc')]
#print(listoffiles)
# Select the files for the three days of interest.
match = [s for s in listoffiles if '20200206' in s or '20200207' in s or '20200208' in s]
#print(match)

# Plot the RS data.
for name in match:
    rs = xr.open_dataset(name)
    ta = rs.ta.values-273.15
    sc = ax[0].scatter(rs.flight_time.values, rs.alt.values, c=ta, cmap=cmap, vmin=10, vmax=25)
    if ta[0,0]>20: # Take only the first value of the ascending RS: we want the surface air temperature.
        ta_surf[kk] = ta[0,0]
        time_surf[kk] = rs.flight_time[0,0].values
    else:
        ta_surf[kk] = np.nan
        time_surf[kk] = np.datetime64('nat')
    kk += 1
    
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.15, 0.02, 0.7]) # Set the position and dimension of the colorbar.
fig.colorbar(sc, shrink=0.5, cax=cbar_ax, label='Air temperature [°C]')

# Read now the TSG: take all the files in the folder.
path2tsg = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/meteor/tsg_aeris/'
listoffiles_tsg = [x for x in glob(path2tsg + '*.nc')]
for name in listoffiles_tsg:
    tsg = xr.open_dataset(name)
    ax[1].plot(tsg.TIME.values, tsg.SSJT.values, '.')

ax[0].set_ylim(0,3000)
ax[1].plot(time_surf,ta_surf,'d')
#ax[1].set_ylim(-5,0)
ax[0].set_ylabel('Height [m]')
ax[1].set_ylabel('Temperature [°C]')
ax[1].set_xlabel('Time [UTC]')
ax[1].legend(['Sea 6Feb','Sea 7Feb','Sea 8Feb','Air'])
plt.subplots_adjust(wspace=0, hspace=0)

plt.savefig('meteor_rs_temp_low_three_days.png', dpi=400)
plt.show()

In [ ]:
# Setup the figure: RH in the lower 3000 m.

fig, ax = plt.subplots(2,1, figsize=(15, 6), sharex=True, gridspec_kw={"height_ratios": [2, 1]})
ax = ax.ravel()

cmap = cm.get_cmap('viridis',8).reversed() # Set here the colormap and the number of colors.
kk = 0
ta_surf = np.zeros(len(match))
time_surf = np.empty(len(match), dtype='datetime64[ns]')

# Open all the three days.

# Radiosounding
path2rs = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/RS_level_1/MET/Vaisala/'
listoffiles = [x for x in glob(path2rs + '*.nc')]
#print(listoffiles)
# Select the files for the three days of interest.
match = [s for s in listoffiles if '20200206' in s or '20200207' in s or '20200208' in s]
#print(match)

# Plot the RS data.
for name in match:
    rs = xr.open_dataset(name)
    ta = rs.ta.values-273.15
    sc = ax[0].scatter(rs.flight_time.values, rs.alt.values, c=rs.rh.values, cmap=cmap, vmin=0.1, vmax=0.9)
    if ta[0,0]>20: # Take only the first value of the ascending RS: we want the surface air temperature.
        ta_surf[kk] = ta[0,0]
        time_surf[kk] = rs.flight_time[0,0].values
    else:
        ta_surf[kk] = np.nan
        time_surf[kk] = np.datetime64('nat')
    kk += 1
    
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.15, 0.02, 0.7]) # Set the position and dimension of the colorbar.
fig.colorbar(sc, shrink=0.5, cax=cbar_ax, label='Air temperature [°C]')

# Add the ceilometer clod base height.
path2ceil = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/meteor/ceilometer/'
listoffiles_ceil = [x for x in glob(path2ceil + '*.nc')]
for name in listoffiles_ceil:
    ceil = xr.open_dataset(name)
    ax[0].plot(ceil.time.values, ceil.cbh.sel(layer=1),'.r')
ax[0].legend(['1st layer cloud base height'], loc=2)


# Read now the TSG: take all the files in the folder.
path2tsg = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/meteor/tsg_aeris/'
listoffiles_tsg = [x for x in glob(path2tsg + '*.nc')]
for name in listoffiles_tsg:
    tsg = xr.open_dataset(name)
    ax[1].plot(tsg.TIME.values, tsg.SSJT.values, '.')

ax[0].set_ylim(0,3000)
ax[1].plot(time_surf,ta_surf,'d')
#ax[1].set_ylim(-5,0)
ax[0].set_ylabel('Height [m]')
ax[1].set_ylabel('Temperature [°C]')
ax[1].set_xlabel('Time [UTC]')
ax[1].legend(['Sea 6Feb','Sea 7Feb','Sea 8Feb','Air'])
plt.subplots_adjust(wspace=0, hspace=0)

plt.savefig('meteor_rs_rh_low_three_days.png', dpi=400)
plt.show()

In [ ]:
# Setup the figure: wind speed in the lower 3000 m.

fig, ax = plt.subplots(2,1, figsize=(15, 6), sharex=True, gridspec_kw={"height_ratios": [2, 1]})
ax = ax.ravel()

cmap = cm.get_cmap('viridis',15) # Set here the colormap and the number of colors.
kk = 0
ta_surf = np.zeros(len(match))
time_surf = np.empty(len(match), dtype='datetime64[ns]')

# Open all the three days.

# Radiosounding
path2rs = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/RS_level_1/MET/Vaisala/'
listoffiles = [x for x in glob(path2rs + '*.nc')]
#print(listoffiles)
# Select the files for the three days of interest.
match = [s for s in listoffiles if '20200206' in s or '20200207' in s or '20200208' in s]
#print(match)

# Plot the RS data.
for name in match:
    rs = xr.open_dataset(name)
    ta = rs.ta.values-273.15
    sc = ax[0].scatter(rs.flight_time.values, rs.alt.values, c=rs.wspd.values, cmap=cmap, vmin=5, vmax=20)
    if ta[0,0]>20: # Take only the first value of the ascending RS: we want the surface air temperature.
        ta_surf[kk] = ta[0,0]
        time_surf[kk] = rs.flight_time[0,0].values
    else:
        ta_surf[kk] = np.nan
        time_surf[kk] = np.datetime64('nat')
    kk += 1
    
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.15, 0.02, 0.7]) # Set the position and dimension of the colorbar.
fig.colorbar(sc, shrink=0.5, cax=cbar_ax, label='Wind speed [m/s]')

# Read now the TSG: take all the files in the folder.
path2tsg = '/media/agostino/TOSHIBA EXT/eurec4a/hackathon_202007/meteor/tsg_aeris/'
listoffiles_tsg = [x for x in glob(path2tsg + '*.nc')]
for name in listoffiles_tsg:
    tsg = xr.open_dataset(name)
    ax[1].plot(tsg.TIME.values, tsg.SSJT.values, '.')

ax[0].set_ylim(0,3000)
ax[1].plot(time_surf,ta_surf,'d')
#ax[1].set_ylim(-5,0)
ax[0].set_ylabel('Height [m]')
ax[1].set_ylabel('Temperature [°C]')
ax[1].set_xlabel('Time [UTC]')
ax[1].legend(['Sea 6Feb','Sea 7Feb','Sea 8Feb','Air'])
plt.subplots_adjust(wspace=0, hspace=0)

plt.savefig('meteor_rs_wspd_low_three_days.png', dpi=400)
plt.show()